In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from copy import copy
import datetime
import pickle
from geopy.geocoders import Nominatim

In [2]:
filename = "xgb_model.sav"
loaded_model = pickle.load(open(filename, 'rb'))

In [3]:
#Sample date

date_list = [15, 5, 2022] #May 15, 2022

year = int(date_list[2])
month = int(date_list[1])
day = int(date_list[0])

my_date = datetime.date(year, month, day)
my_date

#Creating Sample test locations (Took first seven locations of the dataset)

test_locations = {'L1': (40.76793671, -73.98215485),
                  'L2': (40.76560211, -73.96463013),
                  'L3': (40.73856354, -73.98041534),
                  'L4': (40.73115158, -73.9994812),
                  'L5': (40.7639389, -73.97902679),
                  'L6': (40.71008682, -74.00533295),
                  'L7': (40.7199707, -74.01004028)
                  }


In [4]:
#we can get addresses using geolocator
geolocator = Nominatim(user_agent="MyApp")
addresses = []

for key in test_locations:
    location = geolocator.reverse(test_locations[key])
    addresses.append(location.address)

addresses

['59th Street–Columbus Circle, Central Park Outer Loop, Manhattan, New York County, New York, 10023-7692, United States',
 '146, East 65th Street, Manhattan Community Board 8, New York County, New York, 10065, United States',
 '421, 2nd Avenue, Kips Bay, Manhattan Community Board 6, New York County, New York, 10010, United States',
 'New York University, Jones Alley, Manhattan Community Board 2, New York County, New York, 10012-3332, United States',
 'Avenue of the Americas Plaza, West 56th Street, Midtown, Manhattan Community Board 5, New York County, New York, 10019, United States',
 'Wall Street Synagogue, 47, Beekman Street, Financial District, Manhattan Community Board 1, New York County, New York, 10038, United States',
 '377, Greenwich Street, Tribeca, Manhattan Community Board 1, New York County, New York, 10013, United States']

In [5]:
#let's clean up the addresses and form test addresses list
test_addresses = {'L1': '59th Street–Columbus Circle NY',
                  'L2': '146 East 65th Street NY',
                  'L3': '421 2nd Avenue Kips Bay NY',
                  'L4': 'New York University Jones Alley NY',
                  'L5': 'Avenue of the Americas Plaza West 56th Street Midtown NY',
                  'L6': '47 Beekman Street Financial District NY',
                  'L7': '377 Greenwich Street Tribeca NY'    
             }
test_addresses

{'L1': '59th Street–Columbus Circle NY',
 'L2': '146 East 65th Street NY',
 'L3': '421 2nd Avenue Kips Bay NY',
 'L4': 'New York University Jones Alley NY',
 'L5': 'Avenue of the Americas Plaza West 56th Street Midtown NY',
 'L6': '47 Beekman Street Financial District NY',
 'L7': '377 Greenwich Street Tribeca NY'}

In [7]:
def create_guess(points):
    """
    Creates a possible path between all points, returning to the original.
    Input: List of point IDs
    """
    guess = copy(points)
    np.random.shuffle(guess)
    guess.append(guess[0])
    return list(guess)
print(test_locations.keys())
create_guess(list(test_locations.keys()))


dict_keys(['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7'])


['L3', 'L4', 'L2', 'L7', 'L6', 'L1', 'L5', 'L3']

In [9]:
def create_generation(points, population=100):
    """
    Makes a list of guessed point orders given a list of point IDs.
    Input:
    points: list of point ids
    population: how many guesses to make
    """
    generation = [create_guess(points) for _ in range(population)]
    return generation

test_generation = create_generation(list(test_locations.keys()), population=10)
print(test_generation)

[['L6', 'L5', 'L2', 'L7', 'L3', 'L1', 'L4', 'L6'], ['L2', 'L3', 'L7', 'L5', 'L4', 'L1', 'L6', 'L2'], ['L6', 'L2', 'L7', 'L1', 'L4', 'L3', 'L5', 'L6'], ['L7', 'L2', 'L3', 'L5', 'L1', 'L4', 'L6', 'L7'], ['L2', 'L5', 'L3', 'L6', 'L1', 'L7', 'L4', 'L2'], ['L3', 'L1', 'L7', 'L5', 'L6', 'L2', 'L4', 'L3'], ['L1', 'L2', 'L6', 'L7', 'L3', 'L5', 'L4', 'L1'], ['L1', 'L6', 'L5', 'L2', 'L3', 'L7', 'L4', 'L1'], ['L6', 'L7', 'L3', 'L1', 'L5', 'L4', 'L2', 'L6'], ['L2', 'L5', 'L7', 'L4', 'L3', 'L6', 'L1', 'L2']]


In [30]:
def travel_time_between_points(point1_id, point2_id, hour, date, passenger_count = 1, 
                               store_and_fwd_flag = 0, pickup_minute = 0):
    """
    Given two points, this calculates travel between them based on a XGBoost predictive model
    """
    
    model_data = {'passenger_count': passenger_count,
                  'pickup_longitude' : point1_id[1],
                  'pickup_latitude' : point1_id[0],
                  'dropoff_longitude' : point2_id[1],
                  'dropoff_latitude' : point2_id[0],
                  'store_and_fwd_flag' : store_and_fwd_flag,
                  'pickup_month' : my_date.month,
                  'pickup_day' : my_date.day,
                  'pickup_weekday' : my_date.weekday(),
                  'pickup_hour': hour,
                  'pickup_minute' : pickup_minute,
                  'latitude_difference' : point2_id[0] - point1_id[0],
                  'longitude_difference' : point2_id[1] - point1_id[1],
                  'trip_distance' : 0.621371 * 6371 * (abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(point2_id[0] - point1_id[0]) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(point2_id[0] - point1_id[0]) * np.pi / 180) / 2)))))) + \
                                     abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(point2_id[1] - point1_id[1]) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(point2_id[1] - point1_id[1]) * np.pi / 180) / 2)))))))
                 }

    df = pd.DataFrame([model_data], columns=model_data.keys())
    
    pred = np.exp(loaded_model.predict(xgb.DMatrix(df))) - 1
    
    return pred[0]

coordinates = test_locations
print(coordinates)

#travel_time_between_points(coordinates['L3'], coordinates['L6'], 11, my_date)
#coordinates = test_locations
#print(coordinates)


{'L1': (40.76793671, -73.98215485), 'L2': (40.76560211, -73.96463013), 'L3': (40.73856354, -73.98041534), 'L4': (40.73115158, -73.9994812), 'L5': (40.7639389, -73.97902679), 'L6': (40.71008682, -74.00533295), 'L7': (40.7199707, -74.01004028)}


In [31]:
def fitness_score(guess):
    """
    Loops through the points in the guesses order and calculates
    how much distance the path would take to complete a loop.
    Lower is better.
    """
    score = 0
    for ix, point_id in enumerate(guess[:-1]):
        score += travel_time_between_points(coordinates[point_id], coordinates[guess[ix+1]], 11, my_date)
    return score

def check_fitness(guesses):
    """
    Goes through every guess and calculates the fitness score. 
    Returns a list of tuples: (guess, fitness_score)
    """
    fitness_indicator = []
    for guess in guesses:
        fitness_indicator.append((guess, fitness_score(guess)))
    return fitness_indicator

print(check_fitness(test_generation))

[(['L6', 'L5', 'L2', 'L7', 'L3', 'L1', 'L4', 'L6'], 109.67423677444458), (['L2', 'L3', 'L7', 'L5', 'L4', 'L1', 'L6', 'L2'], 122.93982410430908), (['L6', 'L2', 'L7', 'L1', 'L4', 'L3', 'L5', 'L6'], 125.08935642242432), (['L7', 'L2', 'L3', 'L5', 'L1', 'L4', 'L6', 'L7'], 85.55466842651367), (['L2', 'L5', 'L3', 'L6', 'L1', 'L7', 'L4', 'L2'], 102.80579376220703), (['L3', 'L1', 'L7', 'L5', 'L6', 'L2', 'L4', 'L3'], 129.72163581848145), (['L1', 'L2', 'L6', 'L7', 'L3', 'L5', 'L4', 'L1'], 98.91484785079956), (['L1', 'L6', 'L5', 'L2', 'L3', 'L7', 'L4', 'L1'], 101.58901357650757), (['L6', 'L7', 'L3', 'L1', 'L5', 'L4', 'L2', 'L6'], 95.36197471618652), (['L2', 'L5', 'L7', 'L4', 'L3', 'L6', 'L1', 'L2'], 89.0522928237915)]


In [43]:
#test_generation = create_generation(list(test_locations.keys()), population=10)
def get_breeders_from_generation(guesses, take_best_N=10, take_random_N=5, verbose=True, mutation_rate=0.1):
    """
    This sets up the breeding group for the next generation. You have
    to be very careful how many breeders you take, otherwise your
    population can explode. These two, plus the "number of children per couple"
    in the make_children function must be tuned to avoid exponential growth or decline!
    """
    # First, get the top guesses from last time
    fit_scores = check_fitness(guesses)
    sorted_guesses = sorted(fit_scores, key=lambda x: x[1]) # sorts so lowest is first, which we want
    new_generation = [x[0] for x in sorted_guesses[:take_best_N]]
    best_guess = new_generation[0]
    
    if verbose:
        # If we want to see what the best current guess is!
        print(best_guess)
    
    # Second, get some random ones for genetic diversity
    for _ in range(take_random_N):
        ix = np.random.randint(len(guesses))
        new_generation.append(guesses[ix])
        
    # No mutations here since the order really matters.
    # If we wanted to, we could add a "swapping" mutation,
    # but in practice it doesn't seem to be necessary
    
    np.random.shuffle(new_generation)
    return new_generation, best_guess

#test_generation = create_generation(list(test_locations.keys()), population=10)
#new_generation, best_guess =get_breeders_from_generation(test_generation, take_best_N=10, take_random_N=5, verbose=True, mutation_rate=0.1)

def make_child(parent1, parent2):
    
    """ 
    Take some values from parent 1 and hold them in place, then merge in values
    from parent2, filling in from left to right with cities that aren't already in 
    the child. 
    """
    list_of_ids_for_parent1 = list(np.random.choice(parent1, replace=False, size=len(parent1)//2))
    child = [-99 for _ in parent1]
    
    for ix in range(0, len(list_of_ids_for_parent1)):
        child[ix] = parent1[ix]
    for ix, gene in enumerate(child):
        if gene == -99:
            for gene2 in parent2:
                if gene2 not in child:
                    child[ix] = gene2
                    break
    child[-1] = child[0]
    return child

def make_children(old_generation, children_per_couple=1):
    """
    Pairs parents together, and makes children for each pair. 
    If there are an odd number of parent possibilities, one 
    will be left out. 
    
    Pairing happens by pairing the first and last entries. 
    Then the second and second from last, and so on.
    """
    mid_point = len(old_generation)//2
    next_generation = [] 
    
    for ix, parent in enumerate(old_generation[:mid_point]):
        for _ in range(children_per_couple):
            next_generation.append(make_child(parent, old_generation[-ix-1]))
    return next_generation



In [44]:
current_generation = create_generation(list(test_locations.keys()),population=500)
print_every_n_generations = 5

for i in range(100):
    if not i % print_every_n_generations:
        print("Generation %i: "%i, end='')
        #print(len(current_generation))
        is_verbose = True
    else:
        is_verbose = False
    breeders, best_guess = get_breeders_from_generation(current_generation, 
                                                        take_best_N=250, take_random_N=100, 
                                                        verbose=is_verbose)
    current_generation = make_children(breeders, children_per_couple=3)

Generation 0: ['L5', 'L2', 'L3', 'L6', 'L7', 'L4', 'L1', 'L5']
Generation 5: ['L1', 'L5', 'L2', 'L6', 'L7', 'L4', 'L3', 'L1']
Generation 10: ['L2', 'L6', 'L7', 'L4', 'L3', 'L1', 'L5', 'L2']
Generation 15: ['L2', 'L6', 'L7', 'L4', 'L3', 'L1', 'L5', 'L2']
Generation 20: ['L2', 'L6', 'L7', 'L4', 'L3', 'L1', 'L5', 'L2']
Generation 25: ['L2', 'L6', 'L7', 'L4', 'L3', 'L1', 'L5', 'L2']
Generation 30: ['L1', 'L5', 'L2', 'L6', 'L7', 'L4', 'L3', 'L1']
Generation 35: ['L1', 'L5', 'L2', 'L6', 'L7', 'L4', 'L3', 'L1']
Generation 40: ['L1', 'L5', 'L2', 'L6', 'L7', 'L4', 'L3', 'L1']
Generation 45: ['L1', 'L5', 'L2', 'L6', 'L7', 'L4', 'L3', 'L1']
Generation 50: ['L1', 'L5', 'L2', 'L6', 'L7', 'L4', 'L3', 'L1']
Generation 55: ['L1', 'L5', 'L2', 'L6', 'L7', 'L4', 'L3', 'L1']
Generation 60: ['L1', 'L5', 'L2', 'L6', 'L7', 'L4', 'L3', 'L1']
Generation 65: ['L1', 'L5', 'L2', 'L6', 'L7', 'L4', 'L3', 'L1']
Generation 70: ['L1', 'L5', 'L2', 'L6', 'L7', 'L4', 'L3', 'L1']
Generation 75: ['L1', 'L5', 'L2', 'L6', 'L

In [45]:
def evolve_to_solve(current_generation, max_generations, take_best_N, take_random_N,
                    mutation_rate, children_per_couple, print_every_n_generations, verbose=False):
    """
    Takes in a generation of guesses then evolves them over time using our breeding rules.
    Continue this for "max_generations" times.
    Inputs:
    current_generation: The first generation of guesses
    max_generations: how many generations to complete
    take_best_N: how many of the top performers get selected to breed
    take_random_N: how many random guesses get brought in to keep genetic diversity
    mutation_rate: How often to mutate (currently unused)
    children_per_couple: how many children per breeding pair
    print_every_n_geneartions: how often to print in verbose mode
    verbose: Show printouts of progress
    Returns:
    fitness_tracking: a list of the fitness score at each generations
    best_guess: the best_guess at the end of evolution
    """
    fitness_tracking = []
    for i in range(max_generations):
        if verbose and not i % print_every_n_generations and i>0:
            print("Generation %i: "%i, end='')
            #print(len(current_generation))
            print("Current Best Score: ", fitness_tracking[-1])
            is_verbose = True
        else:
            is_verbose = False      
        breeders, best_guess = get_breeders_from_generation(current_generation, 
                                                            take_best_N=take_best_N, take_random_N=take_random_N, 
                                                            verbose=is_verbose, mutation_rate=mutation_rate)
        fitness_tracking.append(fitness_score(best_guess))
        current_generation = make_children(breeders, children_per_couple=children_per_couple)
    
    return fitness_tracking, best_guess

current_generation = create_generation(list(test_locations.keys()),population=500)
fitness_tracking, best_guess = evolve_to_solve(current_generation, 100, 150, 70, 0.5, 3, 5, verbose=True)

Generation 5: Current Best Score:  59.48570251464844
['L7', 'L4', 'L5', 'L1', 'L2', 'L3', 'L6', 'L7']
Generation 10: Current Best Score:  59.48570251464844
['L5', 'L1', 'L2', 'L3', 'L6', 'L7', 'L4', 'L5']
Generation 15: Current Best Score:  59.48570251464844
['L7', 'L4', 'L5', 'L1', 'L2', 'L3', 'L6', 'L7']
Generation 20: Current Best Score:  59.48570251464844
['L7', 'L4', 'L5', 'L1', 'L2', 'L3', 'L6', 'L7']
Generation 25: Current Best Score:  59.48570251464844
['L7', 'L4', 'L5', 'L1', 'L2', 'L3', 'L6', 'L7']
Generation 30: Current Best Score:  59.48570251464844
['L7', 'L4', 'L5', 'L1', 'L2', 'L3', 'L6', 'L7']
Generation 35: Current Best Score:  59.48570251464844
['L7', 'L4', 'L5', 'L1', 'L2', 'L3', 'L6', 'L7']
Generation 40: Current Best Score:  59.48570251464844
['L7', 'L4', 'L5', 'L1', 'L2', 'L3', 'L6', 'L7']
Generation 45: Current Best Score:  59.48570251464844
['L7', 'L4', 'L5', 'L1', 'L2', 'L3', 'L6', 'L7']
Generation 50: Current Best Score:  59.48570251464844
['L7', 'L4', 'L5', '